# EXO

### importer les données json dans votre base de données

In [1]:
import json
import pandas as pd
import pymongo
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost', 27017)
db = client['library']
collection = db['library_collection']

def store(header=None):
    
    with open('repertoire-bibliotheques.json') as file:
        file_data = json.load(file)

    if isinstance(file_data, list): 
        collection.insert_many(file_data)   
    else: 
        collection.insert_one(file_data) 

    client.close()
                           
store()

### combien de bibliotheques se trouve a paris

In [3]:
paris_libraries = collection.count_documents({"fields.commune": "Paris"})
paris_libraries

615

### Faites un update sur le champs "commune" pour le passer en lower case et refaite le compte des bibliotheques a paris

In [4]:
collection.update_many(
  {},
  [{ "$set": { "fields.commune": { "$toLower": "$fields.commune" } } }]
)
paris_libraries = collection.count_documents({"fields.commune": "paris"})
paris_libraries

615

### afficher tout les noms d'établissements situé 95

In [11]:
pipeline = [
        { "$match": { "fields.codepostal": { "$regex": "95[0-9]{3}$" } } }, 
        {"$group": {"_id": {"nom": "$fields.nometablissement", "codepostal": "$fields.codepostal"}}}
]

libraries_in_95 = collection.aggregate(pipeline)

l = list(libraries_in_95)
l

[{'_id': {'nom': 'Institut universitaire de technologie. Bibliothèque',
   'codepostal': '95100'}},
 {'_id': {'nom': "Musée d'Art et d'Histoire Louis Senlecq. Bibliothèque et archives",
   'codepostal': '95290'}},
 {'_id': {'nom': 'Médiathèque intercommunale Georges Pompidou',
   'codepostal': '95320'}},
 {'_id': {'nom': 'Médiathèque municipale Les Temps Modernes',
   'codepostal': '95155'}},
 {'_id': {'nom': 'Bibliothèque du Pays de France', 'codepostal': '95270'}},
 {'_id': {'nom': 'Bibliothèque-Médiathèque Antoine de Saint-Exupéry',
   'codepostal': '95330'}},
 {'_id': {'nom': "Réseau des médiathèques intercommunales du Val Parisis, Val d'Oise",
   'codepostal': '95250'}},
 {'_id': {'nom': "Musée de la Renaissance - château d'Ecouen",
   'codepostal': '95440'}},
 {'_id': {'nom': 'Médiathèque municipale François Mauriac',
   'codepostal': '95190'}},
 {'_id': {'nom': 'Bibliothèque des Dames Gilles', 'codepostal': '95490'}},
 {'_id': {'nom': 'Bibliothèque intercommunale  Anna-Langfus. 

### requeter la bdd pour creer un dictionnaire comportant le nombre de bibliotheque ouvert par jour de la semaine

In [6]:
monday    = collection.count_documents( { "fields.heuresouverture": { "$regex": "lundi|Lundi" } } )
tuesday   = collection.count_documents( { "fields.heuresouverture": { "$regex": "mardi|Mardi" } } )
wednesday = collection.count_documents( { "fields.heuresouverture": { "$regex": "mercredi|Mercredi" } } )
thursday  = collection.count_documents( { "fields.heuresouverture": { "$regex": "jeudi|Jeudi" } } )
friday    = collection.count_documents( { "fields.heuresouverture": { "$regex": "vendredi|Vendredi" } } )
saturday  = collection.count_documents( { "fields.heuresouverture": { "$regex": "samedi|Samedi" } } )
sunday    = collection.count_documents( { "fields.heuresouverture": { "$regex": "dimanche|Dimanche" } } )

dictionnaire = { 
    "monday": monday, 
    "tuesday": tuesday, 
    "wednesday": wednesday, 
    "thursday": thursday, 
    "friday": friday, 
    "saturday": saturday,
    "sunday":sunday
}

dictionnaire

{'monday': 624,
 'tuesday': 493,
 'wednesday': 442,
 'thursday': 394,
 'friday': 839,
 'saturday': 447,
 'sunday': 70}

### certains jours sont bien plus representé que d'autres pourquoi ?

# TP

### le TP consiste a :
  - acquerir de la donnees se mettant a jour assez régulièrement via une api ou du scrapping
    ex: (meteo, bourse, crypto-monaie ect.)
  - stocker cette données dans votre base mongodb rajouter la date precise si ce n'est pas dans vos data
  - mettre en place un outils permetant de visualiser ses data par rapport a la date
    ex:(streamlit, flask ect.)